In [51]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from scipy import signal
from scipy.io import wavfile
from scipy.interpolate import UnivariateSpline
from ipywidgets import interact
import ipywidgets as widgets

matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'


In [39]:
fs, p = wavfile.read('sonido-presion/presion_segmento_1.wav')
fs, b = wavfile.read('sonido-presion/beta_segmento_1.wav')
fs, s_m = wavfile.read('sonido-presion/sonido_segmento_1.wav')
fs, delta = wavfile.read('sonido-presion/delta_segmento_1.wav')

In [55]:

#funcion del sistema de ecuaciones
def f(X, t, params):
    w, x = X
    k, c, delta, beta = params

    gamma = 1e4 #reescaleo
    mu = 1e5

    f_w = -gamma * k / mu * (x + delta / k)
    f_x = gamma * mu * (w - c * x**3 / mu + beta * x / mu )
    return np.array([f_w,f_x])

#método Runge-Kutta 4
def paso_RK(X, f, t, dt, params, orden=4):
    k1 = f(X, t, params)
    k2 = f(X + k1 * dt / 2, t, params)

    if orden == 2:
        X = X + (k1 + k2) / 2 * dt
    elif orden == 4:
        k3 = f(X + k2 * dt / 2, t, params)
        k4 = f(X + k3 * dt, t, params)
        X = X + (k1 + 2 * (k2 + k3) + k4) / 6 * dt
    return X


In [35]:
fs, s_m = wavfile.read('sonido-presion/sonido_segmento_1.wav')
fs, s_s = wavfile.read('sonido-presion/simulacion_segmento_1.wav')

def get_spectrogram(data, sampling_rate, window=1024, overlap=1/1.1,
                    sigma=102.4, fmax=8000, drange=6):
    fu, tu, Sxx = signal.spectrogram(data, sampling_rate, nperseg=window,
                                     noverlap=window*overlap,
                                     window=signal.get_window
                                     (('gaussian', sigma), window),
                                     scaling='spectrum')
    Sxx = np.clip(Sxx, a_min=np.amax(Sxx)/10**drange, a_max=np.amax(Sxx))
    return fu, tu, Sxx

#sonido
fu, tu, Sxx = get_spectrogram(s_m, 44150)
Sxx = np.clip(Sxx, a_min=np.amax(Sxx)/20000000, a_max = np.amax(Sxx))

#simulacion
fu_s, tu_s, Sxx_s = get_spectrogram(s_s[::20], 44150)
Sxx_s = np.clip(Sxx_s, a_min=np.amax(Sxx_s)/20000000, a_max = np.amax(Sxx_s))



In [37]:
#plot
%matplotlib qt
fig, [ax1, ax2] = plt.subplots(2, 1, sharex=True)
ax1.pcolormesh(tu, fu/1000, np.log(Sxx), rasterized=True, shading='auto', 
                  cmap='binary')
ax1.set_ylim([0, 15])
ax2.pcolormesh(tu_s, fu_s/1000, np.log(Sxx_s), rasterized=True, shading='auto', 
                  cmap='binary')
ax2.set_ylim([0, 15])


(0.0, 15.0)

In [48]:
dt = 1/(44150 * 20)
x0 = 0.01
w0 = 0
X0 = np.array([w0, x0])

N = int(len(s_m))
c = 1        
print(f'N = {N}')
ts = np.arange(0, N + 1) * dt
Xs = np.zeros((N + 1, 2))
Xs[0] = X0

N = 38976


In [67]:
import matplotlib.colors as mcolors
colors = [(1,0,0,c) for c in np.linspace(0,1,100)]
cmapred = mcolors.LinearSegmentedColormap.from_list('mycmap', colors, N=5)
colors = [(0,0,1,c) for c in np.linspace(0,1,100)]
cmapblue = mcolors.LinearSegmentedColormap.from_list('mycmap', colors, N=5)

In [58]:
%matplotlib inline
def plot1(k, f0):
    Xs = np.zeros((N + 1, 2))
    Xs[0] = X0
    
    # Integración
    for j in range(N):
        params = [k, c, delta[j] * f0, b]
        Xs[j + 1] = paso_RK(Xs[j], system, ts[j], dt, params, 4)
    
        
    fig, [ax1, ax2] = plt.subplots(2, 1, sharex=True)

    #espectrograma
    #sonido
    fu, tu, Sxx = get_spectrogram(s_m, 44150)
    Sxx = np.clip(Sxx, a_min=np.amax(Sxx)/20000000, a_max = np.amax(Sxx))
    
    #simulacion
    fu_s, tu_s, Sxx_s = get_spectrogram(Xs[:,0][::20], 44150)
    Sxx_s = np.clip(Sxx_s, a_min=np.amax(Sxx_s)/20000000, a_max = np.amax(Sxx_s))

    
    colors = ['green','red','black','blue']
    #ploteo 
    ax1.plot(t,Xs[:,0][:-1]/ max(Xs[:,0][:-1]),alpha = .60, color=colors[3], label = r'simulación')
    ax1.plot(t_m,s_m/max(s_m) ,alpha = .60, color='orange', label = r'sonido')
    ax1.plot(t,b_/ max(b_),alpha = .60, color=colors[1], label = r'$\beta$')
    ax1.plot(t,delta_interp,alpha = .60, color=colors[2], label = r'$\delta$')
    ax1.legend(loc=0)
    ax1.set_ylabel(r'$\omega(t)$', fontsize=16)

    #ploteo espectrograma
    ax2.pcolormesh(tu_s, fu_s/1000, np.log(Sxx_s), rasterized=True, shading='auto', cmap=cmapblue, label = 'simulación')
    ax2.pcolormesh(tu, fu/1000, np.log(Sxx), rasterized=True, shading='auto', cmap=cmapred, label = 'sonido')
    ax2.legend(loc=0)
    ax2.set_xlabel('$Tiempo [s]$', fontsize=14)
    ax2.set_ylabel(r'espectrograma', fontsize=16)
    plt.show()

#params
k = widgets.IntSlider(value=1 , min= 1 , max= 11, step= 1, description='k')
f0 = widgets.FloatSlider(value= 1 , min= 0.1 , max= 2, step= 1e-4, description='f0')

# linkeo el slider con los graficos
output1 = widgets.interactive_output(plot1, {'k': k, 'f0': f0})

# muestro los outputs#
display(k, f0, output1)
#probar d_min 0.56, d_max 0.6

IntSlider(value=1, description='k', max=11, min=1)

FloatSlider(value=1.0, description='f0', max=2.0, min=0.1, step=0.0001)

Output()